In [5]:
from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
from natsort import natsorted

from pathlib import Path
import cv2
import numpy as np
from tqdm import tqdm

# reload notebook automatically after changes to source python files
%load_ext autoreload
%autoreload 2

# change base folder to parent
import os
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
print(os.getcwd())

# for evo ######################################################################
from evo.tools import log
log.configure_logging(verbose=False, debug=False, silent=False)

import pprint
import numpy as np

from evo.tools import plot
import matplotlib.pyplot as plt

# temporarily override some package settings
from evo.tools.settings import SETTINGS
SETTINGS.plot_usetex = False

plot.apply_settings(SETTINGS)
%matplotlib inline

from evo.tools import file_interface
#############################################################################

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/onyx/work_dirs/rjayanti/mast3r-vloc


### Write Poses Camera TUM

In [8]:
from src.datasets.dataset_utils import load_tum_poses, PoseMode
from src.utils.tf_utils import compose_qt_tf, decompose_tf_qt

dataset_root = Path("data/rrc-lab-data/wheelchair-runs-20241220/")
ref_scene_folder = dataset_root / "run-3-wheelchair-query"

ref_poses_file = ref_scene_folder / "poses_lidar_tum.txt"
timestamps, ref_poses = load_tum_poses(ref_poses_file, PoseMode.MAT4x4)

qt_l2c = [0.04150990912955882, -0.05306897521382651, -0.06873946470925439, 0.5053342667867295, -0.4989716676762047, 0.49992869417558417, -0.4957175147273307]
T_l2c = compose_qt_tf(qt_l2c[3:], qt_l2c[:3], in_xyzw=True)

poses_camera_tum = []
for timestamp, tf_w2l in zip(timestamps, ref_poses):
    tf_w2c = tf_w2l @ T_l2c
    qvec_w2c, tvec_w2c = decompose_tf_qt(tf_w2c, in_xyzw=True)
    
    # build TUM format pose as [timestamp] [tx] [ty] [tz] [qx] [qy] [qz] [qw]
    pose_camera_tum = np.concatenate([np.array([timestamp]), tvec_w2c, qvec_w2c])
    poses_camera_tum.append(pose_camera_tum)
    
poses_camera_tum = np.array(poses_camera_tum)
# write poses to file
poses_camera_tum_file = ref_scene_folder / "poses_camera_tum.txt"
np.savetxt(poses_camera_tum_file, poses_camera_tum, fmt='%.9f')

### Read Intrinsics

In [ ]:
# from src.datasets.dataset_utils import read_intrinsics

# data_root = Path("data/rrc-lab-data/wheelchair-runs-20241220/")
# experiment = "run-1-wheelchair-mapping"

# # load intrinsics
# intrinsics_dict = read_intrinsics(data_root / experiment / "intrinsics.txt")
# K_common, W, H = intrinsics_dict['K3x3'], intrinsics_dicts['width'], intrinsics_dicts['height']

# print(intrinsics_dict)

### Plot Trajectories using EVO

In [5]:
# data_root = Path("data/rrc-lab-data/wheelchair-runs-20241220/")

# experiments = [
#     "run-1-wheelchair-mapping",
#     "run-2-wheelchair-query",
#     "run-3-wheelchair-query",
# ]

# tum_mapping = file_interface.read_tum_trajectory_file(data_root / experiments[0] / "poses_tum.txt")
# tum_query_1 = file_interface.read_tum_trajectory_file(data_root / experiments[1] / "poses_tum.txt")
# tum_query_2 = file_interface.read_tum_trajectory_file(data_root / experiments[2] / "poses_tum.txt")

In [6]:
# fig = plt.figure(figsize=(10, 20))
# traj_by_label = {
#     "mapping_run": tum_mapping, 
#     "query_run_1_again": tum_query_2,
#     "query_run_2": tum_query_2,
# }

# plot.trajectories(fig, traj_by_label, plot.PlotMode.xy, plot_start_end_markers=True)
# # fix the plt legend so that only the first and the last are shown in the legend
# handles, labels = plt.gca().get_legend_handles_labels()
# plt.legend([handles[0], handles[-1]], [labels[0], labels[-1]])

# plt.show()